In [49]:
import sys
import pandas as pd
import numpy as np
import shapely
import math

point1 = shapely.geometry.Point(436025, 4.64495e+06) # report location
test_points = [(point1.x + 50, point1.y - 50),
               (point1.x + 50, point1.y + 50),
               (point1.x - 50, point1.y - 50),
               (point1.x - 50, point1.y + 50),
               (point1.x,      point1.y - 50),
               (point1.x,      point1.y + 50),
               (point1.x + 50, point1.y),
               (point1.x - 50, point1.y)
              ]
test_dirs = ["SW",
             "NW",
             "SE",
             "NE",
             "S",
             "N",
             "W",
             "E"
            ]

# This data uses UTM coordinates, so increasing X moves west and
# increasing Y moves north
def get_bearing(row: pd.Series, point2: shapely.geometry.Point, point1_geom='geometry'):
    """A method that returns the number of degrees from due north a line is.

    This line is defined as being from point1 (typically the source point) 
    to point2 (typically the destination).
    
    The point point1 is presumed to come from the row indexed by the string
    'geometry' in the pandas Series passed to this method as row.

    This function is designed to be used in a pandas.DataFrame.apply() call.
    """
    
    point1:shapely.geometry.Point = row['geometry']
    point3:shapely.geometry.Point = shapely.geometry.Point(point1.x, point2.y)
    A:float = point1.distance(point3) # adjacent leg
    B:float = point2.distance(point3) # opposite leg
    C:float = point1.distance(point2) # hypotenuse leg
    # To find the bearing, we need to know the angle from a right triangle
    # defined by point1 (the source point), point2 (the destination point), and
    # point3, the point due north or south of point1 at a distance defined by
    # point2.y.
    #
    # The angle needed is of the leg between point1 and point3 and the leg
    # between point1 and point2.
    theta:float = math.degrees(asin(A/C))
    # Start with bearing of 0 degrees (a.k.a. due north)
    bearing:float = 0.0
    # X is equal, so point 2 is either north or south
    if point2.x == point1.x:
        # if point2.y is bigger, it's north, so no need to modify bearing
        # otherwise, bearing is 180 degrees
        if point2.y < point1.y:
            bearing = 180.0
    # Y is equal, so either due east or due west
    elif point2.y == point1.y:
        # point2.x is bigger, so due west
        if point2.x > point1.x:
            bearing = 270
        # point2.x is smaller, so due east
        else:
            bearing = 90
    # Point2.x is bigger, so Point2 is west of Point1
    elif point2.x > point1.x:
        # Point2.y is bigger, so NW
        if point2.y > point1.y:
            bearing = theta + 270
        # Point2.y is smaller, so SW
        else:
            bearing = theta + 180
    # Point2.x is smaller, so Point2 is east of Point1
    else:
        # Point2.y is bigger, so NE
        if point2.y > point1.y:
            bearing = theta
            # bearing = theta + 90
        # Point2.y is smaller, so SE
        else:
            bearing = theta + 90
    return round(bearing, 2)

for y in range(len(test_points)):
    point2 = shapely.geometry.Point(test_points[y])
    point3 = shapely.geometry.Point(point1.x, point2.y)
    bearing: float = get_bearing(pd.Series([point1], index=['geometry']), point2, 'geometry')
    print(bearing)
    print("-O-" * 10)
print(help(get_bearing))

225.0
-O--O--O--O--O--O--O--O--O--O-
315.0
-O--O--O--O--O--O--O--O--O--O-
135.0
-O--O--O--O--O--O--O--O--O--O-
45.0
-O--O--O--O--O--O--O--O--O--O-
180.0
-O--O--O--O--O--O--O--O--O--O-
0.0
-O--O--O--O--O--O--O--O--O--O-
270
-O--O--O--O--O--O--O--O--O--O-
90
-O--O--O--O--O--O--O--O--O--O-
Help on function get_bearing in module __main__:

get_bearing(row: pandas.core.series.Series, point2: shapely.geometry.point.Point, point1_geom='geometry')
    A method that returns the number of degrees from due north a line is.
    
    This line is defined as being from point1 (typically the source point) 
    to point2 (typically the destination).
    
    The point point1 is presumed to come from the row indexed by the string
    'geometry' in the pandas Series passed to this method as row.
    
    This function is designed to be used in a pandas.DataFrame.apply() call.

None
